# Проверка дат загруженных данных

Проверка дат загруженных данных на пропуски.

In [9]:
import sqlite3
import pandas as pd
from pathlib import Path
from itertools import groupby
from datetime import datetime, timedelta

In [10]:
# Загрузка данных из bd в DF
# Подключаемся к базе данных SQLite
path = Path(fr'c:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day_pj1.db')
conn = sqlite3.connect(path)

# Читаем данные из базы данных в DataFrame
df = pd.read_sql_query("SELECT * FROM Futures", conn)

# Закрываем соединение с базой данных
conn.close()

# Преобразуем все заголовки столбцов к нижнему регистру
df.columns = df.columns.str.lower()

# Выводим первые строки DataFrame
print(df.head())

    tradedate      secid     open      low     high    close  volume  \
0  2015-01-05  RIH5_2015  76930.0  72470.0  78980.0  74600.0  372848   
1  2015-01-06  RIH5_2015  74470.0  71200.0  74610.0  73480.0  319307   
2  2015-01-08  RIH5_2015  73490.0  71000.0  81380.0  79980.0  537469   
3  2015-01-09  RIH5_2015  79950.0  74450.0  81050.0  77650.0  592715   
4  2015-01-12  RIH5_2015  77210.0  73180.0  77550.0  73900.0  440908   

   openposition shortname    lsttrade  
0        751996  RTS-3.15  2015-03-16  
1        731236  RTS-3.15  2015-03-16  
2        751010  RTS-3.15  2015-03-16  
3        865312  RTS-3.15  2015-03-16  
4        897314  RTS-3.15  2015-03-16  


In [11]:
# Преобразуем столбец с датами в формат datetime
df['tradedate'] = pd.to_datetime(df['tradedate'])

# Создаем полный список дат в заданном диапазоне
start_date = df['tradedate'].min()
end_date = df['tradedate'].max()
full_date_range = pd.date_range(start=start_date, end=end_date)

# Находим пропущенные даты
missing_dates = full_date_range[~full_date_range.isin(df['tradedate'])]

# Преобразуем строки в объекты datetime
dates = pd.to_datetime(missing_dates)
# print(missing_dates)

# Удаляем даты, которые приходятся на субботу (день недели 5) и воскресенье (день недели 6)
dates = dates[(dates.dayofweek != 5) & (dates.dayofweek != 6)]
print(dates)

# # Преобразуем обратно в список
# filtered_dates_list = filtered_dates.strftime('%Y-%m-%d').tolist()

# # Выводим отфильтрованный список дат
# print(filtered_dates_list)

DatetimeIndex(['2015-01-07', '2015-02-23', '2015-03-09', '2015-05-01',
               '2015-05-04', '2015-05-11', '2015-06-12', '2015-11-04',
               '2015-12-31', '2016-01-01',
               ...
               '2023-05-01', '2023-05-09', '2023-06-12', '2024-01-01',
               '2024-01-02', '2024-02-23', '2024-03-08', '2024-05-01',
               '2024-05-09', '2024-06-12'],
              dtype='datetime64[ns]', length=103, freq=None)


In [12]:
# Функция для определения разницы между датами
def date_diff(date1, date2):
    return (date2 - date1).days

# Группируем идущие последовательно даты
grouped_dates = []
for key, group in groupby(enumerate(dates), lambda x: x[0] - date_diff(dates[0], x[1])):
    group = list(group)
    if len(group) > 1:
        grouped_dates.append([group[0][1], group[-1][1]])
    else:
        grouped_dates.append([group[0][1]])

# Выводим группы идущих последовательно дат
for group in grouped_dates:
    if len(group) > 1:
        print(f"Группа дат с {group[0].strftime('%Y-%m-%d')} по {group[1].strftime('%Y-%m-%d')}")
    else:
        print(f"Дата {group[0].strftime('%Y-%m-%d')} не имеет соседей")

Дата 2015-01-07 не имеет соседей
Дата 2015-02-23 не имеет соседей
Дата 2015-03-09 не имеет соседей
Дата 2015-05-01 не имеет соседей
Дата 2015-05-04 не имеет соседей
Дата 2015-05-11 не имеет соседей
Дата 2015-06-12 не имеет соседей
Дата 2015-11-04 не имеет соседей
Группа дат с 2015-12-31 по 2016-01-01
Группа дат с 2016-01-07 по 2016-01-08
Дата 2016-02-23 не имеет соседей
Дата 2016-03-08 не имеет соседей
Группа дат с 2016-05-02 по 2016-05-03
Дата 2016-05-09 не имеет соседей
Дата 2016-06-13 не имеет соседей
Дата 2016-11-04 не имеет соседей
Дата 2017-01-02 не имеет соседей
Дата 2017-02-23 не имеет соседей
Дата 2017-03-08 не имеет соседей
Дата 2017-05-01 не имеет соседей
Группа дат с 2017-05-08 по 2017-05-09
Дата 2017-06-12 не имеет соседей
Дата 2017-11-06 не имеет соседей
Группа дат с 2018-01-01 по 2018-01-02
Дата 2018-01-08 не имеет соседей
Дата 2018-02-23 не имеет соседей
Дата 2018-03-08 не имеет соседей
Дата 2018-05-01 не имеет соседей
Дата 2018-05-09 не имеет соседей
Дата 2018-06-12 не

Данные довольно чистые. Не имеют критических пропусков дат.  
Есть пропущенный массив дат в марте 2022 года в связи с СВО, но это не должно повлиять на подготовку данных для ML.